In [1]:
!python.exe -m pip install --upgrade pip
!pip install sgp4
!pip install pyproj
!pip install dask distributed

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [17]:
from sgp4.api import accelerated
print(accelerated)

True


In [18]:
import numpy as np
import dask.bag as db
import pyproj
from dask.distributed import Client
from sgp4.api import Satrec, WGS72
from sgp4.api import jday
from datetime import datetime, timedelta

In [16]:
# Initialize Dask client with memory limits and more workers
client = Client(n_workers=16, threads_per_worker=1, memory_limit='0.5GB')  # This will start a local Dask scheduler and worker
print(client)

<Client: 'tcp://127.0.0.1:57089' processes=16 threads=16, memory=7.45 GiB>


In [19]:
def load_tle_data(tle_file):
    satellites = []
    with open(tle_file, 'r') as file:
        lines = file.readlines()
        for i in range(0, len(lines), 3):
            name = lines[i].strip()
            line1 = lines[i+1].strip()
            line2 = lines[i+2].strip()
            satellites.append([name, line1, line2])
    return satellites
print(load_tle_data('30sats.txt')[1])

['AEROCUBE 12A', '1 43556U 18046C   23055.30616910  .00104379  00000+0  13224-2 0  9990', '2 43556  51.6307  23.8082 0003431 184.7728 175.3232 15.58641168258270']


In [20]:
def get_satellite_position(satellite, times):
    positions = []
    sat = Satrec.twoline2rv(satellite[1], satellite[2], WGS72)
    for t in times:
        jd, fr = jday(2024, 6, 10, 0, 0, 0 + t / 1440.0 * 24)
        e, r, v = sat.sgp4(jd, fr)
        if e == 0:
            positions.append((t, r, v))
            # print(positions)
    return positions
sat1 = get_satellite_position(load_tle_data('30sats.txt')[1], [0]) 
print(sat1)

[]


In [21]:
def ecef2lla(pos_x, pos_y, pos_z):
    ecef = pyproj.Proj(proj="geocent", ellps="WGS84", datum="WGS84")
    lla = pyproj.Proj(proj="latlong", ellps="WGS84", datum="WGS84")
    lona, lata, alta = pyproj.transform(ecef, lla, pos_x, pos_y, pos_z, radians=False)
    return lona, lata, alta

In [22]:
def process_satellite(sat, times):
    positions = get_satellite_position(sat, times)
    pos_array = np.array(positions, dtype=object)
    # Debugging print statement to check pos_array shape
    print(f"pos_array shape: {pos_array.shape}")
    lx = []
    ly = []
    lz = []
    try :
        for l in pos_array[:, 1]:
            lx.append(l[0])
            ly.append(l[1])
            lz.append(l[2])
    except IndexError:
        return []
    lon, lat, alt = ecef2lla(lx, ly, lz)
    return [lon, lat, alt]

In [23]:
def dask_processing(tle_file, times):
    satellites = load_tle_data(tle_file)
    # Optimize number of partitions based on satellite count and memory
    bag = db.from_sequence(satellites, npartitions=200).map(lambda sat: process_satellite(sat, times))
    return bag.compute()

In [24]:
def in_region(lat, lon, region_coords):
    return (region_coords[0][0] <= lat <= region_coords[1][0] and
            region_coords[2][1] <= lon <= region_coords[3][1])

def filter_positions(positions, region_coords):
    return [pos for pos in positions if in_region(pos[8], pos[7], region_coords)]


In [26]:
import time
start_time = time.time()

if __name__ == "__main__":
    tle_file = '30000sats.txt'
    times = np.arange(0, 1440)  # One minute intervals for one day
    region_coords = [(16.66673, 103.58196), (69.74973, -120.64459), (-21.09096, -119.71009), (-31.32309, -147.79778)]
    results = dask_processing(tle_file, times)
    filtered_results = [filter_positions(res, region_coords) for res in results]

    print(f"Number of satellites in region: {len(filtered_results)}")
print("--- %s seconds ---" % (time.time() - start_time))


Number of satellites in region: 8450
--- 32.603928327560425 seconds ---
